In [1]:
import pandas as pd
import numpy as np
from lightfm.data import Dataset
from lightfm import LightFM

/opt/homebrew/Caskroom/miniforge/base/envs/lightfm_python311/lib/python3.11/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
FILE_PATH_MAIN = '[스트롱라이프]최종_데이터_20250924.xlsx'

def load_and_concatenate_user_data(file_path):
    """
    1차와 2차 시트를 로드, no/no. 칼럼을 통일한 후 수직으로 합침.
    """
    
    HEADER_ROW_INDEX = 0
    
    # 1차, 2차 시트 로드 및 헤더 설정
    df_1st = pd.read_excel(file_path, sheet_name='1차', header=0)
    df_2nd = pd.read_excel(file_path, sheet_name='2차', header=0)

    df_1st.columns = df_1st.columns.astype(str).str.strip()
    df_2nd.columns = df_2nd.columns.astype(str).str.strip()
    
    # User ID 칼럼 이름 통일 ('no.' -> 'no')
    col_to_rename = {col: 'no' for col in df_2nd.columns if isinstance(col, str) and col.strip() == 'no.'}
    if col_to_rename:
        df_2nd.rename(columns=col_to_rename, inplace=True)
    
    # 수직으로 concatenate
    df_user_raw = pd.concat([df_1st, df_2nd], ignore_index=True)
    df_user_raw.rename(columns={'no': 'user_id'}, inplace=True)
    
    return df_user_raw.set_index('user_id', drop=False)


def clean_user_ids(df_user_raw):
    """
    user_id (인덱스)에서 결측치 및 유효하지 않은 잔여 행을 제거하여 user_id를 정리.
    """
    # 문자열 'nan'을 실제 결측치(NaN)로 변환
    df_user_raw.index = df_user_raw.index.to_series().replace('nan', np.nan) 
    
    # 인덱스 값을 숫자형으로 변환 가능한지 확인 (오류 시 NaN 처리)
    valid_user_ids_numeric = pd.to_numeric(df_user_raw.index, errors='coerce')

    # 1. user_id가 결측치가 아니고 (NaN이 아니고)
    # 2. user_id가 0보다 큰 값인 (유효한 ID인) 행만 선택
    valid_indices = df_user_raw.index[valid_user_ids_numeric.notna() & (valid_user_ids_numeric > 0)].unique()

    return df_user_raw.loc[valid_indices].copy()

In [3]:
try:
    # 데이터 로드 및 합치기
    df_user_raw = load_and_concatenate_user_data(FILE_PATH_MAIN)
    
    # user_id 정리 전 상태 출력
    print("사용자 특징 데이터 concatenate")
    print("-" * 35)
    print(f"총 사용자 수 (정리 전): {df_user_raw.index.nunique()}명")
    print(f"데이터프레임 크기 (정리 전): {df_user_raw.shape}")
    
    # 3. user_id 정리
    df_user_clean = clean_user_ids(df_user_raw)
    

except Exception as e:
    print(f"❌ 데이터 로드 또는 처리 중 오류 발생: {e}")

사용자 특징 데이터 concatenate
-----------------------------------
총 사용자 수 (정리 전): 1037명
데이터프레임 크기 (정리 전): (1038, 147)


In [4]:
# 파일 경로 정의
FILE_PATH_META = "제품 메타데이터 최종.xlsx" # 제품 메타데이터 파일

# Item ID 칼럼 이름 확정 (첫 번째 칼럼이 product_id라고 가정)
ITEM_ID_COL = 'product_id' 
PROTEIN_COL = 'protein' # 단백질 정량 칼럼

try:
    # 1. Item Feature 데이터 로드 (Header=0 가정)
    df_item_raw = pd.read_excel(FILE_PATH_META, sheet_name='제품 메타데이터 최종', header=0)
    
    # 2. Item ID 칼럼 이름 통일 및 확인
    df_item_raw.rename(columns={df_item_raw.columns[0]: ITEM_ID_COL}, inplace=True)
    
    print("아이템 특징 데이터 로드 완료 (df_item_raw)")
    print("-" * 30)

    # 3. Item Data 핵심 칼럼 확인
    print("\n--- df_item_raw (제품 특징 데이터) 검증 ---")
    
    # Item ID와 단백질 정량 칼럼만 출력하여 확인
    display_item_cols = [ITEM_ID_COL, PROTEIN_COL]
    
    # df_item_raw의 모든 칼럼을 출력하여 칼럼 이름이 제대로 로드되었는지 최종 확인
    print(f"로드된 Item 칼럼 목록: {df_item_raw.columns.tolist()}")
    
    print(df_item_raw[df_item_raw.columns.intersection(display_item_cols)].head())

except Exception as e:
    print(f"❌ Item Data 로드 중 오류 발생: {e}")

아이템 특징 데이터 로드 완료 (df_item_raw)
------------------------------

--- df_item_raw (제품 특징 데이터) 검증 ---
로드된 Item 칼럼 목록: ['product_id', 'brand_name_kor', 'brand_name', 'ingredient_type', 'category', 'sub_category', 'form_factor', 'serving_size', 'serving_unit', 'servings_total', 'calories', 'protein', 'carbs', 'sugars', 'fats', 'Trans Fat', 'Saturated Fat', 'Dietary Fiber', 'ingredients', 'intake_timing', 'product_name', 'sensory_tags', 'functional_tags', 'feature_tags', 'allergens', 'Unnamed: 25']
                            product_id  protein
0                ANIMAL_MEAL_CHOCOLATE     46.0
1            CBUM_MASSGAINER_CHOCOLATE     53.0
2         CBUM_MASSGAINER_COOKIESCREAM     53.0
3              CBUM_MASSGAINER_VANILLA     53.0
4  GASPARI_REALMASS_CHOCOLATEMILKSHAKE     50.0


In [9]:
# 2) 사용자 응답 제품명을 정확한 product_id로 매핑하는 함수 (AC 컬럼 + AD 컬럼 사용)
def normalize_interaction_id(product_name_ac, flavor_ad, mapping_dict=ITEM_FULL_ID_MAP):
    """
    사용자 응답 (제품명과 맛)을 딕셔너리 키로 조합하여 정확한 product_id로 정규화합니다.

    Args:
        product_name_ac (str): AC 컬럼의 제품명 (예: 'BSN 신타-6 웨이')
        flavor_ad (str): AD 컬럼의 맛 (예: '초코맛')
        mapping_dict (dict): {PRODUCT_FLAVOR: product_id} 형태의 매핑 딕셔너리

    Returns:
        str: 매핑된 product_id 또는 정제된 대체 문자열
    """
    if not product_name_ac or not flavor_ad or not mapping_dict:
        return None 
    
    # 1. 두 입력값을 정제하여 딕셔너리 키 형태로 변환: "PRODUCT_FLAVOR"
    product_clean = str(product_name_ac).strip().upper()
    flavor_clean = str(flavor_ad).strip().upper()
    
    # 딕셔너리 검색에 사용할 최종 키 조합
    search_key = f"{product_clean}_{flavor_clean}"
    
    # 2. 딕셔너리에서 정확히 일치하는 키가 있는지 확인
    if search_key in mapping_dict:
        return mapping_dict[search_key]

    # 3. 매핑되지 않은 경우, 두 이름을 합쳐서 공백 및 특수문자를 제거한 문자열을 반환 (안전장치)
    # 두 인자를 합치기 때문에, 베이스 ID 매핑 함수와는 다른 결과를 반환할 수 있습니다.
    combined_name = f"{product_clean}{flavor_clean}"
    return combined_name.replace(' ', '').replace('-', '').replace('.', '').replace('(', '').replace(')', '')


# --- 사용 예시 ---
print("\n--- 개선된 함수 사용 예시 (AC, AD 분리된 경우) ---")

# 예시 1: 엑셀 파일에 '게토레이 파우더_게토레이맛'으로 매핑된 경우
# (가정: GATORADE_POWDER_LEMONLIME이 매핑되어 있다고 가정)
product_in = '게토레이 파우더'
flavor_in = '게토레이맛'
result1 = normalize_interaction_id(product_in, flavor_in)
print(f"('{product_in}', '{flavor_in}') -> {result1}") 
# 예상 결과: GATORADE_POWDER_LEMONLIME

# 예시 2: 엑셀 파일에 없는 제품의 경우
product_in = '새로운 단백질'
flavor_in = '바나나맛'
result2 = normalize_interaction_id(product_in, flavor_in)
print(f"('{product_in}', '{flavor_in}') -> {result2}") 
# 예상 결과: 새로운단백질바나나맛


--- 개선된 함수 사용 예시 (AC, AD 분리된 경우) ---
('게토레이 파우더', '게토레이맛') -> GATORADE_POWDER_LEMONLIME
('새로운 단백질', '바나나맛') -> 새로운단백질바나나맛


In [8]:
# 2. 엑셀 파일을 읽어 데이터프레임 전체를 로드 (모든 컬럼 유지)
FILE_PATH_ALIGN = "uwellnow_product_align.xlsx"
try:
    df_align = pd.read_excel(FILE_PATH_ALIGN)
    
    # 3. 매핑 키 생성 및 딕셔너리 구축
    # 두 컬럼을 조합하여 새로운 키 (Key)를 만듭니다: "PRODUCT_FLAVOR"
    df_align['MAPPING_KEY'] = (df_align['product'].astype(str).str.strip().str.upper() + 
                               '_' + 
                               df_align['flavor'].astype(str).str.strip().str.upper())

    # 4. 매핑 딕셔너리 생성: {PRODUCT_FLAVOR: product_id}
    # Series를 사용하여 딕셔너리를 만듭니다. (중복된 키가 있다면 첫 번째 값을 사용)
    ITEM_FULL_ID_MAP = pd.Series(
        df_align['product_id'].astype(str).str.strip().str.upper().values,
        index=df_align['MAPPING_KEY']
    ).to_dict()

    # 딕셔너리에서 NaN (결측치) 관련 항목은 제거 (선택 사항)
    if 'NAN_NAN' in ITEM_FULL_ID_MAP:
        del ITEM_FULL_ID_MAP['NAN_NAN']
        
    print("✅ 엑셀 파일 로드 및 정확한 ID 매핑 딕셔너리 생성 완료.")
    print(f"**원본 데이터프레임 (df_align)은 {df_align.shape[1]}개의 컬럼으로 메모리에 유지됩니다.**")

except FileNotFoundError:
    print(f"❌ 오류: 파일 경로 '{FILE_PATH_ALIGN}'를 찾을 수 없습니다. 경로를 확인해주세요.")
    ITEM_FULL_ID_MAP = {}
    df_align = pd.DataFrame()

✅ 엑셀 파일 로드 및 정확한 ID 매핑 딕셔너리 생성 완료.
**원본 데이터프레임 (df_align)은 7개의 컬럼으로 메모리에 유지됩니다.**


In [10]:
# --- LightFM 모델링 변수 정의 ---

# 사용자 특징 (User Feature, Multi-Hot Encoding 대상)
OHE_USER_COLS = [
    # Feature Group 1 (기본 정보 및 활동 패턴)
    '3) 성별', 
    '8) 운동 활동 기간', 
    '7) 프로틴, 프리워크아웃, 전해질 음료, 게이너 등 헬스 보충제 2종 이상을 섭취해 보신 경험이 있으신가요?',
    '9) 주에 몇 회 정도 운동을 진행하시나요?(택1)',
    '10) 알러지 또는 민감성분(복수선택 가능)',
    '11) 평소 챙기는 끼니는 어떻게 되나요?(복수선택 가능)',
    '12) 식사 기준으로 운동 시간은 언제인가요?(택 1)',
    '12-1) 일과(수업,업무,일 등) 기준으로 운동 시간은 언제인가요?(택 1)',
    '12-2) 운동을 제외한 일과 중 활동은 어느 정도로 활발한가요?(택 1)',
    '12-3) 시간 기준으로 운동 시작 시간이 언제인가요?(택 1)',
    # Feature Group 2 (영향 요인 및 고려 항목) - 모든 보충제 관련 고려 항목 포함
    '13-3) 프로틴의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '13-4) 프로틴의 효과에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '13-5) 브랜드에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '13-6) 유명인(선수 또는 전문가)의 사용 여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '13-7) 지인의 사용여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)',
    '14-3) 프리워크아웃의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '14-4) 프리워크아웃의 효과에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '14-5) 브랜드에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '14-6) 유명인(선수 또는 전문가)의 사용 여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '14-7) 지인의 사용여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)',
    '15-3) 전해질 음료의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '15-4) 전해질 음료의 효과에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '15-5) 브랜드에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '15-6) 유명인(선수 또는 전문가)의 사용 여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '15-7) 지인의 사용여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)',
    '16-3) 게이너의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '16-4) 게이너의 효과에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '16-5) 브랜드에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '16-6) 유명인(선수 또는 전문가)의 사용 여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '16-7) 지인의 사용여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)',
    '17-3) 해당 보충제의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '17-4) 해당 보충제의 효과에서 고려한 점은 무엇인가요?', '17-5) 브랜드에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '17-6) 유명인(선수 또는 전문가)의 사용 여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)', '17-7) 지인의 사용여부 또는 추천에서 고려한 세부 항목을 선택해주세요 (복수선택 가능)',
]

# B. 아이템 특징 (Item Feature, Multi-Hot Encoding 대상)
OHE_ITEM_COLS = ['ingredient_type', 'category', 'flavor', 'sensory_tags']

# C. 상호작용 (Interaction, One-Hot Encoding 대상)
# '가장 마음에 들었던 제품'을 Item ID로 사용하고, '재구매 의사'를 가중치로 사용합니다.
INTERACTION_WEIGHT_COLS = [
    ('13-9) [프로틴] 선택하신 제품중에 가장 종합적으로 마음에 들었던 제품 1개만 선택해주세요 (택 1)', '13-10) 마음에 들었던 제품이 어떤 맛인가요? (복수선택 가능)', '13-17) 해당 프로틴에 대한 재구매 의사는 어느 정도인가요?'),
    ('14-9) [프리워크아웃] 선택하신 제품중에 가장 종합적으로 마음에 들었던 제품 1개만 선택해주세요 (택 1)', '14-10) 마음에 들었던 제품이 어떤 맛인가요? (복수선택 가능)', '14-17) 해당 프리워크아웃에 대한 재구매 의사는 어느 정도인가요?'),
    ('15-9) [전해질 음료(BCAA, 이온음료)] 선택하신 제품중에 가장 종합적으로 마음에 들었던 제품 1개만 선택해주세요 (택 1)', '15-10) 마음에 들었던 제품이 어떤 맛인가요? (복수선택 가능)', '15-17) 해당 전해질 음료에 대한 재구매 의사는 어느 정도인가요?'),
    ('16-9) [게이너] 선택하신 제품중에 가장 종합적으로 마음에 들었던 제품 1개만 선택해주세요 (택 1)', '16-10) 마음에 들었던 제품이 어떤 맛인가요? (복수선택 가능)', '16-17) 해당 게이너에 대한 재구매 의사는 어느 정도인가요?'),
    ('17-9) 종합적으로 가장 마음에 들었던 제품 1개를 작성해 주세요', '17-10) 마음에 들었던 제품이 어떤 맛인가요?', '17-17) 해당 제품에 대한 재구매 의사는 어느 정도인가요?'),
]

In [11]:
# df_user_clean의 인덱스 이름 정리 (오류 해결)
if df_user_clean.index.name == 'user_id':
    df_user_clean.index.name = None

try:
    # ---------------------------------------------------------------------------------
    # 🚨 디버깅을 위해 OHE_ITEM_COLS에 어떤 컬럼이 있는지 확인합니다.
    print(f"DEBUG: OHE_ITEM_COLS 목록: {OHE_ITEM_COLS}")
    print("-" * 30)
    
    # 단백질 정량 처리 및 이산화 (Binning)
    df_item_raw[PROTEIN_COL] = pd.to_numeric(df_item_raw[PROTEIN_COL], errors='coerce')
    df_item_raw.dropna(subset=[PROTEIN_COL, ITEM_ID_COL], inplace=True)
    bins = [0, 15, 25, df_item_raw[PROTEIN_COL].max() + 1]
    labels = ['protein_low', 'protein_mid', 'protein_high']
    df_item_raw['protein_bin'] = pd.cut(df_item_raw[PROTEIN_COL], bins=bins, labels=labels, right=False)
    
    # Item Feature Long 포맷 생성
    selected_ohe_cols = [col for col in OHE_ITEM_COLS + ['protein_bin'] if col in df_item_raw.columns]
    
    df_item_features = df_item_raw.melt(
        id_vars=[ITEM_ID_COL], 
        value_vars=selected_ohe_cols
    )
    
    # 🚨 결측치(NaN)를 'NONE' 피처로 대체하여 데이터 손실을 방지합니다.
    df_item_features['value'] = df_item_features['value'].fillna('NONE') 
    # 결측치 처리 후에는 모든 행이 유효하므로, .dropna()는 제거합니다.
    
    # 피처 이름 생성: '컬럼이름_값' (예: 'Category_프로틴', 'protein_bin_protein_mid')
    df_item_features['feature'] = df_item_features['variable'].astype(str) + '_' + df_item_features['value'].astype(str).str.strip()
    
    # 최종 정리
    df_item_features = df_item_features[[ITEM_ID_COL, 'feature']].drop_duplicates()
    
    print(f"DEBUG: Item Features 총 개수 (생성된 피처 종류): {len(df_item_features['feature'].unique())}")
    print(f"DEBUG: Item Features 행 개수 (상호작용): {len(df_item_features)}")
    
    print("✅ Item Feature (정량 포함) 전처리 완료.")
# ---------------------------------------------------------------------------------
# ---------------------------------------------------------------------------------
    ## 6단계: Interaction Matrix 소스 데이터 구축 (랭킹 데이터 추가 통합)
    
    # 기존의 INTERACTION_WEIGHT_COLS 및 새로운 RANKING_COLS_MAP을 모두 사용합니다.
    # (INTERACTION_WEIGHT_COLS는 (제품명, 맛, 가중치) 구조를 유지한다고 가정)
    # 우선 전해질 음료는 ELECTROLYTE 로 정의함
    
    RANKING_COLS_MAP = [
        # (순위 컬럼, 가중치 컬럼, 제품군 태그, 첫 번째 항목 컬럼)
        ('13-2) 프로틴 선택 시 가장 영향을 많이 받은 요소들의 순위를 매겨주세요', '13-17) 해당 프로틴에 대한 재구매 의사는 어느 정도인가요?  ', 'PROTEIN', '13-3) 프로틴의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)'),
        ('14-2) 프리워크아웃 선택 시 가장 영향을 많이 받은 요소들의 순위를 매겨주세요', '14-17) 해당 프리워크아웃에 대한 재구매 의사는 어느 정도인가요?  ', 'PREWORKOUT', '14-3) 프리워크아웃의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)'),
        ('15-2) 전해질 음료(BCAA, 이온음료)선택 시 가장 영향을 많이 받은 요소들의 순위를 매겨주세요', '15-17) 해당 전해질 음료에 대한 재구매 의사는 어느 정도인가요?  ', 'ELECTROLYTE', '15-3) 전해질 음료의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)'),
        ('16-2) 게이너 선택 시 가장 영향을 많이 받은 요소들의 순위를 매겨주세요', '16-17) 해당 게이너에 대한 재구매 의사는 어느 정도인가요?  ', 'GAINER', '16-3) 게이너의 영양성분을 고려한 세부 항목을 선택해주세요 (복수선택 가능)')
    ]
    MAX_RANK_COUNT = 7 
    # 순위의 역순으로 가중치를 부여하는 함수
    def rank_to_weight(rank_value, max_rank=MAX_RANK_COUNT):
        rank_value = pd.to_numeric(rank_value, errors='coerce')
        if pd.isna(rank_value):
            return np.nan
        # 1순위 -> 7, 7순위 -> 1
        return (max_rank + 1) - rank_value 
    
    df_interactions_list = []
    
    # --- 1. 제품명 + 맛 기반 상호작용 (기존 로직: 랭킹 제거) ---
    for item_col, flavor_col, weight_col in INTERACTION_WEIGHT_COLS:
        temp_df = df_user_clean[['user_id', item_col, flavor_col, weight_col]].copy()
        
        temp_df.rename(columns={item_col: 'product', 
                                flavor_col: 'flavor', 
                                weight_col: 'weight'}, 
                       inplace=True)
        
        temp_df.dropna(subset=['product', 'flavor'], inplace=True)
        
        # 제품명/맛 정규화 및 item_id 생성
        temp_df['item_id'] = temp_df.apply(lambda row: normalize_interaction_id(row['product'], row['flavor']), axis=1)
        temp_df.drop(columns=['product', 'flavor'], inplace=True)
        
        # 🚨 상호작용 가중치 10배 증폭 적용
        temp_df['weight'] = temp_df['weight'] * 10
        
        df_interactions_list.append(temp_df)
    
    
    # --- 2. 랭킹 응답 기반 상호작용 (제거됨 - 7단계로 이동) ---
    
    df_interactions = pd.concat(df_interactions_list, ignore_index=True)
    
    df_interactions['weight'] = pd.to_numeric(df_interactions['weight'], errors='coerce')
    
    # Item Feature에 존재하는 유효한 Item ID만 남기고 필터링
    valid_items = df_item_raw[ITEM_ID_COL].unique()
    df_interactions = df_interactions[df_interactions['item_id'].isin(valid_items)]
    
    df_interactions.dropna(subset=['item_id', 'weight'], inplace=True)
    
    # Item ID와 User ID 쌍의 중복 제거 (가장 높은 가중치를 남김)
    df_interactions.sort_values(by='weight', ascending=False, inplace=True)
    df_interactions.drop_duplicates(subset=['user_id', 'item_id'], keep='first', inplace=True)
    
    print("✅ Interaction Matrix 소스 데이터 준비 완료. (랭킹 데이터 User Feature로 이동)")
    
    # ---------------------------------------------------------------------------------
    ## 7단계: User Feature Matrix 구축 (★ 랭킹 데이터 통합)
    df_user_features_list = []
    
    # --- 1. 일반 User Feature 처리 (기존 로직) ---
    df_user_features_ohe = df_user_clean[['user_id'] + OHE_USER_COLS].melt(
        id_vars='user_id', var_name='question', value_name='feature_value'
    ).dropna(subset=['feature_value'])
    
    df_user_features_ohe['feature_value'] = df_user_features_ohe['feature_value'].astype(str).str.split(r'[,/]')
    df_user_features_ohe = df_user_features_ohe.explode('feature_value')
    df_user_features_ohe['feature'] = df_user_features_ohe['question'].astype(str) + '_' + df_user_features_ohe['feature_value'].astype(str).str.strip()
    df_user_features_ohe['weight'] = 1.0 # OHE 피처는 가중치 1.0 유지
    df_user_features_list.append(df_user_features_ohe[['user_id', 'feature', 'weight']].drop_duplicates())
    
    
    # --- 2. 랭킹 데이터 User Feature로 통합 (새로운 로직) ---
    for rank_col, weight_col, product_tag, feature_start_col in RANKING_COLS_MAP:
        if rank_col in df_user_clean.columns:
            
            df_rank_user_feat = df_user_clean[['user_id', rank_col]].copy()
            df_rank_user_feat.rename(columns={rank_col: 'rank_str'}, inplace=True)
            
            # 1. 랭킹 응답 문자열을 각 순위(1, 2, 3...)별로 분리하여 explode
            df_rank_user_feat['rank_list'] = df_rank_user_feat['rank_str'].astype(str).str.strip().apply(list)
            df_rank_user_feat = df_rank_user_feat.explode('rank_list')
            
            # 2. 응답의 순서(Index)를 계산 (1, 2, 3...)
            df_rank_user_feat['feature_index'] = df_rank_user_feat.groupby('user_id').cumcount() 
            
            # 3. Item Feature와 동일한 방법으로 컬럼 헤더 이름 목록 추출
            all_cols = df_user_clean.columns.tolist()
            try:
                start_idx = all_cols.index(feature_start_col)
                feature_cols_list = all_cols[start_idx : start_idx + MAX_RANK_COUNT] 
            except ValueError:
                continue
                
            index_to_col = {i: col for i, col in enumerate(feature_cols_list)}
            
            # 4. Feature 이름 생성 (제품군_RANK_항목헤더)
            df_rank_user_feat['feature_header'] = df_rank_user_feat['feature_index'].map(index_to_col)
            df_rank_user_feat['feature'] = product_tag.upper() + '_RANK_' + df_rank_user_feat['feature_header'].astype(str).str.upper()
            
            # 5. 가중치 계산 (순위 역변환을 가중치로 사용)
            df_rank_user_feat['weight'] = df_rank_user_feat['rank_list'].apply(rank_to_weight) 
            
            df_rank_user_feat.dropna(subset=['feature', 'weight'], inplace=True)
            
            # 최종 User Feature 데이터: user_id, feature, weight
            df_user_features_list.append(df_rank_user_feat[['user_id', 'feature', 'weight']].drop_duplicates())
    
    # --- 3. 최종 User Feature 결합 ---
    df_user_features = pd.concat(df_user_features_list, ignore_index=True)
    df_user_features = df_user_features.drop_duplicates(subset=['user_id', 'feature'], keep='first')
    
    
    print("✅ User Feature Matrix 소스 데이터 준비 완료. (랭킹 가중치 통합)")
    # ---------------------------------------------------------------------------------
    print("\n--- 디버깅: 피처 데이터 상태 확인 ---")
    print(f"User Features 총 개수 (Unique): {len(df_user_features['feature'].unique())}")
    # User Feature의 상위 10개 피처와 해당 가중치 확인
    print("User Features (상위 5개):")
    print(df_user_features.head(5))
    
    print(f"\nItem Features 총 개수 (Unique): {len(df_item_features['feature'].unique())}")
    # Item Feature의 상위 10개 피처와 해당 Item ID 확인
    print("Item Features (상위 5개):")
    print(df_item_features.head(5))

    # ---------------------------------------------------------------------------------
    ## 8단계: LightFM Dataset 구축 및 Train/Test 분리 (안정화된 수동 분리)
    # 1. Dataset 객체 초기화 및 fit (이전과 동일)
    dataset = Dataset()
    dataset.fit(
        users=df_user_clean.index.unique(),
        items=df_item_raw[ITEM_ID_COL].unique(),
        user_features=df_user_features['feature'].unique(),
        item_features=df_item_features['feature'].unique()
    )
    
    # 2. Total Interaction 행렬 구축 (총 상호작용 개수 확인용)
    (interactions_all, sample_weights_all) = dataset.build_interactions(
        (row['user_id'], row['item_id'], row['weight'])
        for index, row in df_interactions.iterrows()
    )
    
    # 3. 데이터프레임 인덱스 기반 수동 분리
    total_indices = np.arange(len(df_interactions))
    np.random.seed(42)
    np.random.shuffle(total_indices)
    
    test_size = int(len(df_interactions) * 0.2)
    train_indices = total_indices[test_size:]
    test_indices = total_indices[:test_size]
    
    # 4. Train/Test 데이터프레임 분리 (인덱스 순서 유지)
    df_train_interactions = df_interactions.iloc[train_indices].copy()
    df_test_interactions = df_interactions.iloc[test_indices].copy()
    
    # 5. Train/Test 행렬 및 가중치 행렬 재구축 (오류 방지 핵심)
    # 분리된 데이터프레임을 사용하여 build_interactions를 호출하면 순서 불일치가 발생하지 않음
    (interactions_train, weights_train) = dataset.build_interactions(
        (row['user_id'], row['item_id'], row['weight']) 
        for index, row in df_train_interactions.iterrows()
    )
    
    (interactions_test, weights_test) = dataset.build_interactions(
        (row['user_id'], row['item_id'], row['weight']) 
        for index, row in df_test_interactions.iterrows()
    )
    
    # 6. Feature Matrix 구축 (동일)
    user_features_matrix = dataset.build_user_features(
        (row['user_id'], {row['feature']: row['weight']}) for index, row in df_user_features.iterrows()
    )
    item_features_matrix = dataset.build_item_features(
        (row[ITEM_ID_COL], [row['feature']]) for index, row in df_item_features.iterrows()
    )
    

    print("\n--- LightFM 행렬 크기 ---")
    print(f"Interaction Matrix (Total): {interactions_all.shape}")
    print(f"Interaction Matrix (Train): {interactions_train.shape} / Non-zero: {interactions_train.getnnz()}")
    print(f"Interaction Matrix (Test): {interactions_test.shape} / Non-zero: {interactions_test.getnnz()}")
    
    # ---------------------------------------------------------------------------------
    ## 9단계: 모델 학습 (Train Set 사용)

    model = LightFM(
        loss='warp', 
        no_components=15,
        learning_rate=0.03, 
        user_alpha=0,
        item_alpha=0,
        random_state=42
    )


    model.fit(
        interactions_train, 
        sample_weight=weights_train, # COO 형식으로 전달
        user_features=user_features_matrix, 
        item_features=item_features_matrix,
        epochs=60, 
        num_threads=4,
    )

    print("\n\nLightFM 하이브리드 추천 모델 학습 완료")

except Exception as e:
    print(f"❌ 최종 모델 구축 중 오류 발생: {e}")

DEBUG: OHE_ITEM_COLS 목록: ['ingredient_type', 'category', 'flavor', 'sensory_tags']
------------------------------
DEBUG: Item Features 총 개수 (생성된 피처 종류): 90
DEBUG: Item Features 행 개수 (상호작용): 489
✅ Item Feature (정량 포함) 전처리 완료.
✅ Interaction Matrix 소스 데이터 준비 완료. (랭킹 데이터 User Feature로 이동)
✅ User Feature Matrix 소스 데이터 준비 완료. (랭킹 가중치 통합)

--- 디버깅: 피처 데이터 상태 확인 ---
User Features 총 개수 (Unique): 448
User Features (상위 5개):
   user_id   feature  weight
0      2.0  3) 성별_남성     1.0
1      4.0  3) 성별_여성     1.0
2      5.0  3) 성별_여성     1.0
3      6.0  3) 성별_여성     1.0
4      7.0  3) 성별_여성     1.0

Item Features 총 개수 (Unique): 90
Item Features (상위 5개):
                            product_id              feature
0                ANIMAL_MEAL_CHOCOLATE  ingredient_type_주원료
1            CBUM_MASSGAINER_CHOCOLATE  ingredient_type_주원료
2         CBUM_MASSGAINER_COOKIESCREAM  ingredient_type_주원료
3              CBUM_MASSGAINER_VANILLA  ingredient_type_주원료
4  GASPARI_REALMASS_CHOCOLATEMILKSHAKE  ingredient_ty

In [12]:
from lightfm.evaluation import precision_at_k, auc_score

# Precision@3 측정
# 학습 데이터와 사용자/아이템 특징을 함께 사용하여 평가합니다.
test_precision = precision_at_k(
    model, 
    interactions_test, 
    user_features=user_features_matrix, 
    item_features=item_features_matrix, 
    k=3
).mean()

print(f"Test Set Precision@3: {test_precision:.4f}")

# AUC Score 측정 (모델의 전반적인 순위 예측 능력을 평가)
test_auc = auc_score(
    model, 
    interactions_test, 
    user_features=user_features_matrix, 
    item_features=item_features_matrix
).mean()

print(f"Test Set AUC Score: {test_auc:.4f}")

Test Set Precision@3: 0.2429
Test Set AUC Score: 0.9691


In [13]:
"""
overfitting 판단용 코드
"""

# 훈련 데이터(Train Set)에 대한 Precision@3 측정
train_precision = precision_at_k(
    model,
    interactions_train, # 👈 interactions_test 대신 interactions_train 사용
    user_features=user_features_matrix,
    item_features=item_features_matrix,
    k=3
).mean()

# 훈련 데이터(Train Set)에 대한 AUC Score 측정
train_auc = auc_score(
    model,
    interactions_train, # 👈 interactions_test 대신 interactions_train 사용
    user_features=user_features_matrix,
    item_features=item_features_matrix
).mean()

print(f"Train Set Precision@3: {train_precision:.4f}")
print(f"Train Set AUC Score: {train_auc:.4f}")

Train Set Precision@3: 0.2497
Train Set AUC Score: 0.9789
